- The file contains an adjacency list representation of an undirected weighted graph with 200 vertices labeled 1 to 200. Each row consists of the node tuples that are adjacent to that particular vertex along with the length of that edge. For example, the 6th row has 6 as the first entry indicating that this row corresponds to the vertex labeled 6. The next entry of this row "141,8200" indicates that there is an edge between vertex 6 and vertex 141 that has length 8200. The rest of the pairs of this row indicate the other vertices adjacent to vertex 6 and the lengths of the corresponding edges.


- Your task is to run Dijkstra's shortest-path algorithm on this graph, using 1 (the first vertex) as the source vertex, and to compute the shortest-path distances between 1 and every other vertex of the graph. If there is no path between a vertex v and vertex 1, we'll define the shortest-path distance between 1 and v to be 1000000.


- You should report the shortest-path distances to the following ten vertices, in order: 7,37,59,82,99,115,133,165,188,197. You should encode the distances as a comma-separated string of integers. So if you find that all ten of these vertices except 115 are at distance 1000 away from vertex 1 and 115 is 2000 distance away, then your answer should be 1000,1000,1000,1000,1000,2000,1000,1000,1000,1000. Remember the order of reporting DOES MATTER, and the string should be in the same order in which the above ten vertices are given. The string should not contain any spaces. Please type your answer in the space provided.


- IMPLEMENTATION NOTES: This graph is small enough that the straightforward O(mn) time implementation of Dijkstra's algorithm should work fine. OPTIONAL: For those of you seeking an additional challenge, try implementing the heap-based version. Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

### Heap implementation

In [12]:
'''
    Reference : https://interactivepython.org/runestone/static/pythonds/Trees/BinaryHeapImplementation.html
    Heap class:
        
        Node : each node is a tuple, (distance, node_name)
        __init__ :  
        
'''

class BinHeap(object):
    
    def __init__(self) :
        
        self.heap_list = [(0, None)]
        self.current_size = 1

    def insert(self, new_node) :
        '''
            Insert the node at the end of the heaplist.
            Then use __pecup__ function to make the node to its right place
            The self.heap_list[i][0] is the key
        '''
        self.heap_list.append(new_node)
        self.current_size += 1
        self.__perc_up__(new_node, self.current_size - 1)
    
    def __perc_up__(self, new_node, cursor):
        
        if (self.heap_list[cursor][0] < self.heap_list[cursor/2][0]) and (cursor > 1):
            self.heap_list[cursor/2], self.heap_list[cursor] = self.heap_list[cursor], self.heap_list[cursor/2]
            cursor = cursor/2
            self.__perc_up__(new_node, cursor)
            
    def del_min(self):
        '''
            First : always delete the first element and return it
            Second : make the last element in list to the first
            Third : push down this element
        '''
        min_element = self.heap_list[1]
        self.heap_list[1], self.heap_list[-1] = self.heap_list[-1], self.heap_list[1]
        del(self.heap_list[-1])
        self.current_size = self.current_size - 1
        self.__push__(cursor = 1)
        return min_element
        
    def __push__(self, cursor):
        
        min_child_cursor = self.__min_child__(cursor)
        if (min_child_cursor is not None) and (self.heap_list[min_child_cursor][0] < self.heap_list[cursor][0]):
            self.heap_list[cursor],self.heap_list[min_child_cursor] = self.heap_list[min_child_cursor],self.heap_list[cursor]
            cursor = min_child_cursor
            self.__push__(cursor = min_child_cursor -1)
            
    def __min_child__(self, cursor):
        
        if (2 * cursor) > (self.current_size - 1) :
            return None
        elif (2 * cursor + 1) > (self.current_size - 1):
            return 2 * cursor
        else:
            if self.heap_list[2 * cursor][0] > self.heap_list[2 * cursor + 1][0]:
                return 2 * cursor + 1
            else:
                return 2 * cursor
            
    def check_unique(self):
        '''
           Since the element in heap can not repeat.
           Check if there is repeat
           If there is repeat, Raise repeat exception
        '''
        ele_name = []
        for ele in self.heap_list:
            ele_name.append(ele[1])
        if len(ele_name) != len(set(ele_name)):
            raise ValueError('There are repeat values in list')
            
    def decrease_key(self, key, node_name):
        '''
            Because of new froniter, some of the distance will decrease
            keep the invariant
        '''
        i = 0
        while i < self.current_size:
            if self.heap_list[i][1] == node_name:
                self.heap_list[i] = (key, self.heap_list[i][1])
                self.__perc_up__(self.heap_list[i], i)
                break  
            i += 1
            
    def is_empty(self):
        if self.current_size < 2:
            return True
        else:
            return False
    
    def get_key(self,node_name):
        for ele in self.heap_list:
            if ele[1] == node_name:
                return ele[0]

#### test for heap

In [13]:
bin_heap = BinHeap()
bin_heap.insert((1, 1))
bin_heap.insert((9, 4))
bin_heap.insert((5, 5))
bin_heap.insert((2, 2))
bin_heap.insert((7, 7))
# bin_heap.insert((3, 3))
bin_heap.del_min()
bin_heap.decrease_key(4,4)
bin_heap.del_min()
bin_heap.heap_list

[(0, None), (4, 4), (7, 7), (5, 5)]

In [14]:
bin_heap.get_key(7)

7

### Dijkastra

In [30]:
'''
    distance_dict = {end : dis, ...}
    road_dict = {end : [node1, node2, ...], ...}
'''

def dijkstra(source, d_dict):
    
    distance_dict = {}
    priority_queue = BinHeap()

    distance_dict[source] = 0
    for ele in d_dict[source]:
        priority_queue.insert(ele)

    node_process = priority_queue.del_min()
    distance_dict[node_process[1]] = node_process[0]

    while priority_queue.current_size > 1:

        # get the node tuples in dijkstra dict
        unprocessed = [ele for ele in d_dict[node_process[1]] if ele[1] not in distance_dict.keys()]

        # print unprocessed
        # find intersection of what in heap and what in unprocessed
        # keep the invariant
        node_in_heap = [ele[1] for ele in priority_queue.heap_list]

        for ele in unprocessed:
            if ele[1] in node_in_heap:
                min_dis = min([priority_queue.get_key(ele[1]),
                              distance_dict[node_process[1]] + ele[0]])
                priority_queue.decrease_key(min_dis, ele[1])
            else:
                ele = ((ele[0] + distance_dict[node_process[1]]), ele[1])
                priority_queue.insert(ele)
                
        # print priority_queue.heap_list
        node_process = priority_queue.del_min()
        distance_dict[node_process[1]] = node_process[0]

    return distance_dict

#### The second and third comes from coursera example

In [11]:
source = 1
edges = {1: [(1, 2), (6, 3)], 
         2: [(1, 1), (1, 5)],
         3: [(6, 1), (3, 5), (2, 4)],
         4: [(2, 3), (4, 5), (2, 6), (2, 7)],
         5: [(3, 3), (1, 2), (4, 4), (5, 6), (1, 7)],
         6: [(5, 5), (2, 4), (2, 8)],
         7: [(1, 5), (2, 4), (10, 8)],
         8 :[(2, 6), (10, 7)]}
dijkstra(source, edges)

{1: 0, 2: 1, 3: 5, 4: 5, 5: 2, 6: 7, 7: 3, 8: 9}

In [31]:
source = 1
edges = {1: [(9, 2), (2, 3), (6, 4)], 
         2: [(4, 6)],
         3: [(3, 4), (1, 5)],
         4: [(2, 2), (7, 6), (9, 7)],
         5: [(6, 8), (1, 4)],
         6: [(1, 7), (5, 9)],
         7: [(1, 9), (5, 8)],
         8 :[(5, 9)]}
dijkstra(source, edges)

{1: 0, 2: 6, 3: 2, 4: 4, 5: 3, 6: 10, 7: 11, 8: 9, 9: 12}

In [32]:
source = 1
edges = {1: [(2, 3), (3, 2)], 
         2: [(4, 4)],
         3: [(1, 2), (2, 4), (3, 5)],
         4: [(2, 5), (1, 6)],
         5: [(2, 6)]}
dijkstra(source, edges)

{1: 0, 2: 3, 3: 2, 4: 4, 5: 5, 6: 5}

### Read txt file

In [33]:
'''
    dijkastra will look like [[1,(),()],[2,(),()],[],...]
'''

def main():
    
    answer = []
    
    dijkastra_dict = {}
    with open('/Users/boyazhou/Desktop/Algorithm/Assignment5/dijkstraData.txt') as file:
        for line in file:
            single_list = []
            for ele in line.strip().split('\t')[1:]:
                single_tuple = ()
                single_tuple = (int(ele.split(',')[1]),int(ele.split(',')[0]))
                single_list.append(single_tuple)
            dijkastra_dict[int(line.strip().split('\t')[0])] = single_list
            
    goal = [7,37,59,82,99,115,133,165,188,197]
    source = 1
#     goal = [1]
#     source = 59
    re_list = dijkstra(source, dijkastra_dict)
    for g in goal:
        answer.append(re_list[g])
    
    return answer

In [34]:
main()

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]